In [ ]:
import os
import re
import numpy as np
import pandas as pd
from PIL import Image
from segment_anything import sam_model_registry, SamPredictor

In [ ]:
# Load SAM Model
model_type = "vit_h"  # Model type can be "vit_h", "vit_l", or "vit_b"
sam = sam_model_registry[model_type](checkpoint="./sam_vit_h_4b8939.pth")
predictor = SamPredictor(sam)

In [ ]:
def get_identifier(filename: str) -> str:
    """
    Extracts a unique identifier from a filename using a regular expression pattern.
    
    Args:
        filename (str): The filename of the image (e.g., 'A1_03_1_1_Phase Contrast_001.tif').
    
    Returns:
        str: A unique identifier based on the first two parts and last three digits of the filename.
             If the filename does not match the expected pattern, returns None.
    """
    match = re.match(r"^([A-D]\d+_\d+).*_(\d{3})\.tif$", filename)
    return f"{match.group(1)}_{match.group(2)}" if match else None

def split_mask(mask: np.array) -> tuple:
    """
    Splits a mask into two regions: head (first one-third) and tail (remaining two-thirds).

    Args:
        mask (np.array): Binary mask of the fish.

    Returns:
        tuple: Two binary masks, (head_mask, tail_mask).
    """
    split_index = mask.shape[1] // 3  # Compute the index for one-third of the image width

    head_mask = np.zeros_like(mask)  # Mask for the head (first one-third)
    tail_mask = np.zeros_like(mask)  # Mask for the tail (remaining two-thirds)

    head_mask[:, :split_index] = mask[:, :split_index]  # First one-third (head)
    tail_mask[:, split_index:] = mask[:, split_index:]  # Remaining two-thirds (tail)

    return head_mask, tail_mask

def process_images(input_dir: str, output_dir: str, 
                   save_masks: bool = False, save_masked_images: bool = False) -> pd.DataFrame:
    """
    Processes pairs of 'Phase Contrast' and 'GFP' images to generate a binary mask using SAM, 
    calculate GFP intensity within the mask, and optionally save mask and masked images.

    Steps:
    1. Matches pairs of 'Phase Contrast' and 'GFP' images based on unique identifiers.
    2. Uses SAM (Segment Anything Model) to create a binary mask of the zebrafish in the 'Phase Contrast' image.
    3. Calculates the mean and total GFP intensity within the masked zebrafish region in the corresponding GFP image.
    4. Splits the primary mask into head and tail regions and calculates GFP intensities for each.
    5. Saves the results to a DataFrame and optionally outputs masks and masked GFP images.

    Args:
        input_dir (str): Directory containing the input TIF images.
        output_dir (str): Directory to save output files (e.g., masks and results).
        save_masks (bool): If True, saves generated masks as PNG files for visual confirmation.
        save_masked_images (bool): If True, saves the masked GFP images to output directory.

    Returns:
        pd.DataFrame: A DataFrame containing GFP intensity data (mean and total) for each image pair.
    """
    phase_mask_dir = os.path.join(output_dir, "phase_masks")
    if save_masks:
        os.makedirs(phase_mask_dir, exist_ok=True)

    phase_contrast_files = {}
    gfp_files = {}

    for file_name in os.listdir(input_dir):
        if file_name.endswith(".tif"):
            identifier = get_identifier(file_name)
            if identifier:
                if "Phase Contrast" in file_name:
                    phase_contrast_files[identifier] = file_name
                elif "GFP" in file_name:
                    gfp_files[identifier] = file_name

    gfp_intensity_results = []

    for i, (identifier, phase_file) in enumerate(phase_contrast_files.items()):
        if identifier in gfp_files:
            try:
                phase_path = os.path.join(input_dir, phase_file)
                gfp_path = os.path.join(input_dir, gfp_files[identifier])
                
                # Load Phase Contrast image
                phase_image = Image.open(phase_path)
                phase_np = np.array(phase_image, dtype=np.uint16)
                phase_np = (phase_np / phase_np.max() * 255).astype(np.uint8)
                phase_rgb = np.stack([phase_np] * 3, axis=-1)

                predictor.set_image(phase_rgb)

                # Generate primary mask
                input_point = np.array([[phase_rgb.shape[1] // 2, phase_rgb.shape[0] // 2]])
                input_label = np.array([1])
                masks, scores, _ = predictor.predict(
                    point_coords=input_point,
                    point_labels=input_label,
                    multimask_output=False
                )
                primary_mask = masks[0]

                # Save mask if required
                if save_masks:
                    mask_output_path = os.path.join(phase_mask_dir, f"mask_{os.path.splitext(phase_file)[0]}.png")
                    Image.fromarray((primary_mask * 255).astype(np.uint8)).save(mask_output_path)

                # Split mask into head and tail
                head_mask, tail_mask = split_mask(primary_mask)

                # Load GFP image
                gfp_image = Image.open(gfp_path)
                gfp_np = np.array(gfp_image, dtype=np.uint16)

                # Calculate GFP intensities
                gfp_values_within_fish = gfp_np[primary_mask > 0]
                mean_gfp_intensity = gfp_values_within_fish.mean() if gfp_values_within_fish.size > 0 else 0
                total_gfp_intensity = gfp_values_within_fish.sum()

                gfp_values_within_head = gfp_np[head_mask > 0]
                mean_gfp_head = gfp_values_within_head.mean() if gfp_values_within_head.size > 0 else 0
                total_gfp_head = gfp_values_within_head.sum()

                gfp_values_within_tail = gfp_np[tail_mask > 0]
                mean_gfp_tail = gfp_values_within_tail.mean() if gfp_values_within_tail.size > 0 else 0
                total_gfp_tail = gfp_values_within_tail.sum()

                # Save masked GFP images
                if save_masked_images:
                    gfp_masked = gfp_np * primary_mask
                    output_path = os.path.join(output_dir, f"masked_{identifier}.png")
                    Image.fromarray(gfp_masked.astype(np.uint16)).save(output_path)

                gfp_intensity_results.append({
                    "Identifier": identifier,
                    "Mean_GFP_Intensity": mean_gfp_intensity,
                    "Total_GFP_Intensity": total_gfp_intensity,
                    "Mean_GFP_Head": mean_gfp_head,
                    "Total_GFP_Head": total_gfp_head,
                    "Mean_GFP_Tail": mean_gfp_tail,
                    "Total_GFP_Tail": total_gfp_tail
                })

            except Exception as e:
                print(f"Error processing {identifier}: {e}")

        if (i + 1) % 5 == 0:
            print(f"Processed {i + 1}/{len(phase_contrast_files)} image pairs.")

    results_df = pd.DataFrame(gfp_intensity_results)
    csv_output_path = os.path.join(output_dir, "gfp_intensity_results.csv")
    results_df.to_csv(csv_output_path, index=False)
    
    print(f"Results saved to {csv_output_path}") 
    return results_df

In [ ]:
# Define paths
input_dir = r"C:\Daten\MoIm\Images"
output_dir = r"C:\Daten\MoIm\Output"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Run the function
gfp_intensity_results_df = process_images(input_dir, output_dir, save_masks=False, save_masked_images=False)